# Medical Insurance Cost Prediction

**Dataset:** https://www.kaggle.com/datasets/mosapabdelghany/medical-insurance-cost-dataset/data  
**Goal:** Predict medical insurance charges (costs) using patient demographic and health information. This notebook is built to run on Kaggle's default environment without installing additional packages.

## 1 — Imports and Version Check

We import the commonly available libraries on Kaggle and print their versions so you can confirm compatibility with the Kaggle runtime. No `pip install` commands are used.

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
import plotly.express as px

# optional xgboost import — available on Kaggle in most runtimes; handle gracefully if absent
try:
    import xgboost as xgb
    _XGBOOST_AVAILABLE = True
except Exception:
    _XGBOOST_AVAILABLE = False

print('Python:', sys.version.splitlines()[0])
print('numpy :', np.__version__)
print('pandas :', pd.__version__)
print('matplotlib :', plt.__version__)
print('seaborn :', sns.__version__)
print('scikit-learn :', sklearn.__version__)
print('tensorflow :', tf.__version__)
print('plotly :', px.__version__)
print('xgboost available :', _XGBOOST_AVAILABLE)

## 2 — Load Dataset

We load the CSV from the Kaggle input path. If the Kaggle input is not present (for local runs), the cell will try to load `./insurance.csv` if present.

In [ ]:
kaggle_path = '/kaggle/input/medical-insurance-cost-dataset/insurance.csv'
local_path = './insurance.csv'

if os.path.exists(kaggle_path):
    data_path = kaggle_path
elif os.path.exists(local_path):
    data_path = local_path
else:
    data_path = None

if data_path is None:
    raise FileNotFoundError('Could not find insurance.csv at the Kaggle input path or local path. On Kaggle make sure the dataset is added to the notebook.
')

print('Loading dataset from:', data_path)
df = pd.read_csv(data_path)

print('
Shape:', df.shape)
display(df.head())
print('
Info:')
display(df.info())
print('
Summary statistics:')
display(df.describe(include='all'))

## 3 — Data Cleaning & Feature Engineering

We check for missing values, treat outliers where appropriate, and encode categorical variables. We also add engineered features such as BMI categories and smoker interactions.

In [ ]:
# Check missing values
missing = df.isnull().sum()
print('Missing values per column:
', missing)

# Make a copy for processing and show initial distribution of 'charges'
df_proc = df.copy()
print('
Charges distribution (summary):')
display(df_proc['charges'].describe())

# Visual check for outliers in charges
plt.figure(figsize=(8,4))
sns.boxplot(x=df_proc['charges'])
plt.title('Boxplot of charges (original)')
plt.show()

# Create a log-transformed target to reduce skewness (we keep original target for interpretation)
df_proc['log_charges'] = np.log1p(df_proc['charges'])
print('
Log-transformed charges summary:')
display(df_proc['log_charges'].describe())

# Create BMI categories
def bmi_category(bmi):
    if bmi < 18.5:
        return 'underweight'
    elif bmi < 25:
        return 'normal'
    elif bmi < 30:
        return 'overweight'
    else:
        return 'obese'

df_proc['bmi_cat'] = df_proc['bmi'].apply(bmi_category)

# Binary encoding for 'sex' and 'smoker' (male:1, female:0; smoker:1, no:0)
df_proc['sex_male'] = (df_proc['sex'] == 'male').astype(int)
df_proc['smoker_flag'] = (df_proc['smoker'] == 'yes').astype(int)

# One-hot encode 'region' using get_dummies for transparency now (we will also show pipeline-based encoding later)
df_region = pd.get_dummies(df_proc['region'], prefix='region')
df_proc = pd.concat([df_proc, df_region], axis=1)

# Smoker interaction features
df_proc['smoker_bmi'] = df_proc['smoker_flag'] * df_proc['bmi']
df_proc['smoker_age'] = df_proc['smoker_flag'] * df_proc['age']

print('After feature engineering — sample rows:')
display(df_proc.head())

### Before-and-after summaries
We show how the number of columns changed and a quick view of the engineered values.

In [ ]:
print('Original columns count:', len(df.columns))
print('Processed columns count:', len(df_proc.columns))
display(df_proc[['charges','log_charges','bmi','bmi_cat','sex_male','smoker_flag','smoker_bmi']].head())

## 4 — Exploratory Data Analysis (9 plots + 1 interactive)

We present a grid of 9 static plots and one interactive Plotly visualization. Interpretations follow each block.

In [ ]:
# Prepare numeric features for pairplot and correlation
numeric_cols = ['age','bmi','children','charges']
sns.set(style='whitegrid')
fig, axes = plt.subplots(3,3, figsize=(16,12))

# 1 Distribution of charges
sns.histplot(df_proc['charges'], kde=True, ax=axes[0,0], color='tab:blue')
axes[0,0].set_title('Distribution of Charges')

# 2 Scatter: age vs charges
sns.scatterplot(data=df_proc, x='age', y='charges', hue='smoker', ax=axes[0,1])
axes[0,1].set_title('Age vs Charges (colored by smoker)')

# 3 Scatter: bmi vs charges
sns.scatterplot(data=df_proc, x='bmi', y='charges', hue='smoker', ax=axes[0,2])
axes[0,2].set_title('BMI vs Charges')

# 4 Boxplot: charges by smoker
sns.boxplot(data=df_proc, x='smoker', y='charges', ax=axes[1,0])
axes[1,0].set_title('Charges by Smoker')

# 5 Barplot: mean charges by region
region_means = df_proc.groupby('region')['charges'].mean().reset_index()
sns.barplot(data=region_means, x='region', y='charges', ax=axes[1,1])
axes[1,1].set_title('Mean Charges by Region')

# 6 Pairplot for numeric features (use a small sample for speed)
sample_for_pair = df_proc[numeric_cols].sample(min(500, len(df_proc)), random_state=1)
sns.pairplot(sample_for_pair)
axes[1,2].axis('off')

# 7 Correlation heatmap
corr = df_proc[numeric_cols].corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', ax=axes[2,0])
axes[2,0].set_title('Correlation Heatmap')

# 8 Violin plot: children vs charges
sns.violinplot(data=df_proc, x='children', y='charges', ax=axes[2,1])
axes[2,1].set_title('Children vs Charges (violin)')

# 9 Density plot of log_charges (helps with skew)
sns.kdeplot(df_proc['log_charges'], ax=axes[2,2], fill=True, color='tab:green')
axes[2,2].set_title('Density of log(1+charges)')

plt.tight_layout()
plt.show()

# Interactive plot: Plotly scatter (age vs charges)
fig_px = px.scatter(df_proc, x='age', y='charges', color='smoker', hover_data=['bmi','children','region'], title='Interactive: Age vs Charges (smoker)')
fig_px.update_traces(marker=dict(size=6))
fig_px.show()

**Interpretation (high level):**
- `smoker` has a strong effect on `charges` - smokers pay much higher costs.
- `age` and `bmi` positively correlate with charges but with substantial spread.
- `children` shows smaller differences.
- Log transform reduces skew in `charges` and can stabilize training.

## 5 — Prepare Data for Modeling

We create feature and target arrays, split into train/test, and construct a preprocessing pipeline using `ColumnTransformer`. The pipeline scales numeric features and one-hot encodes categorical features.

In [ ]:
# Choose features (use engineered ones and original meaningful columns)
feature_cols = ['age', 'bmi', 'children', 'sex_male', 'smoker_flag', 'smoker_bmi', 'smoker_age', 'bmi_cat', 'region']
target_col = 'charges'
X = df_proc[feature_cols].copy()
y = df_proc[target_col].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('Train shape:', X_train.shape, 'Test shape:', X_test.shape)

# Preprocessing: numeric and categorical
numeric_features = ['age','bmi','children','smoker_bmi','smoker_age']
categorical_features = ['sex_male','smoker_flag','bmi_cat','region']

numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop'
)

# Fit preprocessor on training data and transform
preprocessor.fit(X_train)
X_train_trans = preprocessor.transform(X_train)
X_test_trans = preprocessor.transform(X_test)

print('Transformed train shape:', X_train_trans.shape)

# Save preprocessor pipeline for later use
preproc_path = '/kaggle/working/preprocessor_pipeline.joblib'
joblib.dump(preprocessor, preproc_path)
print('Preprocessor saved to:', preproc_path)

## 6 — Build and Train TensorFlow Neural Network

We train a simple feed-forward network with early stopping and monitor MAE. We'll evaluate using MAE, RMSE and R².

In [ ]:
tf.random.set_seed(42)
input_dim = X_train_trans.shape[1]
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(input_dim,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse', metrics=['mae'])
model.summary()

# Callbacks
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Fit the model (use modest epochs so it runs quickly on Kaggle)
history = model.fit(X_train_trans, y_train, validation_split=0.2, epochs=200, batch_size=32, callbacks=[early_stop], verbose=2)

# Save the trained model
model_path = '/kaggle/working/tf_insurance_model'
model.save(model_path)
print('TensorFlow model saved to:', model_path)

### Training Curves
We plot training and validation loss and MAE to inspect for overfitting.

In [ ]:
# Plot training history
hist = history.history
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(hist['loss'], label='train_loss')
plt.plot(hist['val_loss'], label='val_loss')
plt.legend(); plt.title('Loss (MSE)')

plt.subplot(1,2,2)
plt.plot(hist['mae'], label='train_mae')
plt.plot(hist['val_mae'], label='val_mae')
plt.legend(); plt.title('MAE')
plt.show()

## 7 — Evaluation on Test Set
We compute MAE, RMSE and R² on the test set and provide prediction plots and residual analysis.

In [ ]:
# Predictions
y_pred = model.predict(X_test_trans).reshape(-1)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
print(f'TF Model — MAE: {mae:.2f}, RMSE: {rmse:.2f}, R2: {r2:.4f}')

# Predicted vs Actual
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Actual Charges')
plt.ylabel('Predicted Charges')
plt.title('Predicted vs Actual — TensorFlow')
plt.show()

# Residual analysis
residuals = y_test - y_pred
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
sns.histplot(residuals, kde=True, bins=30)
plt.title('Residuals Distribution')

plt.subplot(1,2,2)
plt.scatter(y_pred, residuals, alpha=0.6)
plt.axhline(0, color='r', linestyle='--')
plt.xlabel('Predicted')
plt.ylabel('Residuals')
plt.title('Residuals vs Predicted')
plt.show()

## 8 — Optional: XGBoost Benchmark
We train a simple XGBoost regressor for comparison if `xgboost` is available in the environment.

In [ ]:
if _XGBOOST_AVAILABLE:
    print('Training XGBoost regressor (this may take a short while)...')
    xgb_model = xgb.XGBRegressor(n_estimators=200, max_depth=6, learning_rate=0.05, random_state=42, n_jobs=4)
    xgb_model.fit(X_train_trans, y_train, eval_set=[(X_test_trans, y_test)], early_stopping_rounds=10, verbose=False)
    y_pred_xgb = xgb_model.predict(X_test_trans)
    mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
    rmse_xgb = mean_squared_error(y_test, y_pred_xgb, squared=False)
    r2_xgb = r2_score(y_test, y_pred_xgb)
    print(f'XGBoost — MAE: {mae_xgb:.2f}, RMSE: {rmse_xgb:.2f}, R2: {r2_xgb:.4f}')
    # Feature importances — map back to column names where possible
    try:
        # Recover feature names from the preprocessor
        ohe = preprocessor.named_transformers_['cat'].named_steps['onehot']
        cat_cols = categorical_features[:2] + list(ohe.get_feature_names_out(categorical_features[2:]))
        num_cols = numeric_features
        feature_names = num_cols + list(cat_cols)
    except Exception:
        # Fallback to numeric-length placeholders
        feature_names = [f'f{i}' for i in range(X_train_trans.shape[1])]
    importances = xgb_model.feature_importances_
    top_idx = np.argsort(importances)[-10:][::-1]
    print('Top XGBoost features:')
    for i in top_idx:
        name = feature_names[i] if i < len(feature_names) else f'feat_{i}'
        print(f'  {name}: {importances[i]:.4f}')
else:
    print('XGBoost is not available in this environment. Skipping XGBoost benchmark.')

## 9 — Save Models & Pipeline Paths
We already saved the TensorFlow model and the preprocessing pipeline earlier; display their locations again.

In [ ]:
print('Preprocessor pipeline:', preproc_path)
print('TensorFlow model folder:', model_path)
# Optionally save a lightweight sklearn wrapper or the model weights (already saved above)
# joblib.dump(model, '/kaggle/working/tf_model_sklearn_wrapper.joblib')  # not recommended for TF models

## 10 — Summary & Next Steps
